# TASK3: DECISION TREE

In [116]:
import pandas as pd
import numpy as np
data=pd.read_csv('datasetB.csv')

In [145]:
data.head()
data1=data

In [146]:
data1.head()

fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0              0                 1            0               0          0   
1              1                 2            0               0          0   
2              1                 1            0               0          0   
3              2                 0            2               0          0   
4              0                 1            0               0          0   

   free sulfur dioxide  total sulfur dioxide  density  PH  sulphates  alcohol  \
0                    0                     0        2   2          0        0   
1                    1                     0        1   1          0        0   
2                    0                     0        2   1          0        0   
3                    0                     0        2   1          0        0   
4                    0                     0        2   2          0        0   

   quality  
0        1  
1        1  
2        1  
3        1  
4        1

In [118]:
data['fixed acidity'].value_counts()

1    801
0    574
2    204
3     20
Name: fixed acidity, dtype: int64

In [119]:
data['quality'].value_counts()

1    1319
2     217
0      63
Name: quality, dtype: int64

In [120]:
data['fixed acidity'].unique()

array([0, 1, 2, 3], dtype=int64)

In [121]:
data['quality'].unique()

array([1, 2, 0], dtype=int64)

In [122]:
#calculate the entropy of a dataset
def entropy(target_col):
    elements,counts = np.unique(target_col,return_counts = True)
    entropy = np.sum([(-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)) for i in range(len(elements))])
    return entropy

In [123]:
#calculate the information gain of a dataset
def InfoGain(data,split_attribute_name,target_name="quality"):   
    total_entropy = entropy(data[target_name])
    vals,counts= np.unique(data[split_attribute_name],return_counts=True)
    Weighted_Entropy = np.sum([(counts[i]/np.sum(counts))*entropy(data.where(data[split_attribute_name]==vals[i]).dropna()[target_name]) for i in range(len(vals))])
    Information_Gain = total_entropy - Weighted_Entropy
    return Information_Gain

In [124]:
#the main ID3 function
def ID3(data,originaldata,features,target_attribute_name="quality",parent_node_class = None):
    
    if len(np.unique(data[target_attribute_name])) <= 1:
        return np.unique(data[target_attribute_name])[0]
    
    #If the dataset has less than 10 elements, return the mode target feature value in the original dataset
    elif len(data)<=10:
        return np.unique(originaldata[target_attribute_name])[np.argmax(np.unique(originaldata[target_attribute_name],return_counts=True)[1])]
    
    elif len(features) ==0:
        return parent_node_class
    
    #If none of the above holds true, grow the tree!
    
    else:
        parent_node_class = np.unique(data[target_attribute_name])[np.argmax(np.unique(data[target_attribute_name],return_counts=True)[1])]
        
        #Select the feature which best splits the dataset
        item_values = [InfoGain(data,feature,target_attribute_name) for feature in features] 
        #Return the information gain values for the features in the dataset
        best_feature_index = np.argmax(item_values)
        best_feature = features[best_feature_index]
        
        #Create the tree structure. The root gets the name of the feature (best_feature) with the maximum information
        #gain in the first run
        tree = {best_feature:{}}
        
        
        #Remove the feature with the best inforamtion gain from the feature space
        features = [i for i in features if i != best_feature]
        
        #Grow a branch under the root node for each possible value of the root node feature
        
        for value in np.unique(data[best_feature]):
            value = value
            #Split the dataset along the value of the feature with the largest information gain and therwith create sub_datasets
            sub_data = data.where(data[best_feature] == value).dropna()
            
            #Call the ID3 algorithm for each of those sub_datasets with the new parameters --> Here the recursion comes in!
            subtree = ID3(sub_data,originaldata,features,target_attribute_name,parent_node_class)
            
            #Add the sub tree, grown from the sub_dataset to the tree under the root node
            tree[best_feature][value] = subtree
            
        return(tree)    

In [125]:
def predict(query,tree,default = 1): 
    for key in list(query.keys()):
        if key in list(tree.keys()):
            try:
                result = tree[key][query[key]] 
            except:
                return default
            result = tree[key][query[key]]
            if isinstance(result,dict):
                return predict(query,result)
            else:
                return result


In [126]:
def train_test_split(dataset):
    training_data = dataset.iloc[:1200].reset_index(drop=True)
    testing_data = dataset.iloc[1200:].reset_index(drop=True)
    return training_data,testing_data

training_data = train_test_split(data)[0]
testing_data = train_test_split(data)[1]

In [127]:
def test(data,tree):
    #Create new query instances by simply removing the target feature column from the original dataset and 
    #convert it to a dictionary
    queries = data.iloc[:,:-1].to_dict(orient = "records")
    
    #Create a empty DataFrame in whose columns the prediction of the tree are stored
    predicted = pd.DataFrame(columns=["predicted"]) 
    
    #Calculate the prediction accuracy
    for i in range(len(data)):
        predicted.loc[i,"predicted"] = predict(queries[i],tree) 
    return predicted
#print('The prediction accuracy is: ',(np.sum(predicted["predicted"] == data["quality"])/len(data))*100,'%')

In [128]:
tree = ID3(training_data,training_data,training_data.columns[:-1])

In [129]:
predicted=test(testing_data,tree)


In [130]:
predicted.head()

predicted
0        1.0
1        1.0
2        2.0
3        1.0
4        1.0

In [131]:
predicted['predicted'].value_counts()

1.0    387
2.0     10
0.0      2
Name: predicted, dtype: int64

# DECISION TREE USING SKLEARN

In [133]:
from sklearn.tree import DecisionTreeClassifier
sk=DecisionTreeClassifier(min_samples_leaf=10)
sk.fit(data.iloc[:,0:11],data.iloc[:,11])
y_pred_sk=list(sk.predict(data.iloc[:,0:11]))

In [134]:
for i in range(3):

    print(str(len(data[data.iloc[:,11]==i]))+"  "+str(y_pred_sk.count(i)))

63  0
1319  1535
217  64


In [136]:
y=data['quality']
d=data.drop('quality',axis=1)
print(y.head())
print(d.head())

0    1
1    1
2    1
3    1
4    1
Name: quality, dtype: int64
   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0              0                 1            0               0          0   
1              1                 2            0               0          0   
2              1                 1            0               0          0   
3              2                 0            2               0          0   
4              0                 1            0               0          0   

   free sulfur dioxide  total sulfur dioxide  density  PH  sulphates  alcohol  
0                    0                     0        2   2          0        0  
1                    1                     0        1   1          0        0  
2                    0                     0        2   1          0        0  
3                    0                     0        2   1          0        0  
4                    0                     0        2   2          0

In [137]:
y.value_counts()

1    1319
2     217
0      63
Name: quality, dtype: int64

In [138]:
d['alcohol'].value_counts()

0    747
1    617
2    215
3     20
Name: alcohol, dtype: int64

In [142]:
data.head()

fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0              0                 1            0               0          0   
1              1                 2            0               0          0   
2              1                 1            0               0          0   
3              2                 0            2               0          0   
4              0                 1            0               0          0   

   free sulfur dioxide  total sulfur dioxide  density  PH  sulphates  alcohol  \
0                    0                     0        2   2          0        0   
1                    1                     0        1   1          0        0   
2                    0                     0        2   1          0        0   
3                    0                     0        2   1          0        0   
4                    0                     0        2   2          0        0   

   quality  
0        1  
1        1  
2        1  
3        1  
4        1

# Cross Validation with 3-Folds Classifier

In [143]:
from sklearn.model_selection import KFold
from sklearn import metrics

In [144]:
kfold=KFold(n_splits=3)
split=kfold.split(range(len(data)))
acc1=0
prec1=0
rec1=0

acc2=0
prec2=0
rec2=0
for train_index,test_index in split:
    training_data=pd.DataFrame(data.iloc[train_index,:])
    testing_data=pd.DataFrame(data.iloc[test_index,:])
    tree = ID3(training_data,training_data,training_data.columns[:-1])
    result=test(testing_data,tree)
    
    cm1=metrics.confusion_matrix(testing_data['quality'],result['predicted'])
    print("confusion matrix of created model",cm1)
    print('\n')
    total=len(testing_data)
    correct=cm1[0][0]+cm1[1][1]+cm1[2][2]
    acc1+=correct/total
    prec_temp=0
    for i in range(3):
        temp=(cm1[0][i]+cm1[1][i]+cm1[2][i])
        if temp!=0:
            prec_temp+=cm1[i][i]/temp    
        prec_temp/=3
        prec1+=prec_temp
    rec_temp=0
    for i in range(3):
        temp=(cm1[i][0]+cm1[i][1]+cm1[i][2])
        if temp!=0:
             rec_temp+=cm1[i][i]/temp    
        rec_temp/=3
        rec1+=rec_temp
    
    
    
    sk=DecisionTreeClassifier(min_samples_leaf=10)
    sk.fit(training_data.iloc[:,0:11],training_data.iloc[:,11])
    y_pred_sk=list(sk.predict(testing_data.iloc[:,0:11]))
    cm2=metrics.confusion_matrix(testing_data['quality'],y_pred_sk)
    print("confusion matrix of sklearn model",cm2)
    print('\n')
    total=len(testing_data)
    correct=cm2[0][0]+cm2[1][1]+cm2[2][2]
    acc2+=correct/total
    prec_temp=0
    for i in range(3):
        temp=(cm2[0][i]+cm2[1][i]+cm2[2][i])
        if temp!=0:
            prec_temp+=cm2[i][i]/temp    
        prec_temp/=3
        prec2+=prec_temp
    rec_temp=0
    for i in range(3):
        temp=(cm2[i][0]+cm2[i][1]+cm2[i][2])
        if temp!=0:
             rec_temp+=cm2[i][i]/temp    
        rec_temp/=3
        rec2+=rec_temp
        
        
acc1=acc1/3
prec1=prec1/3
rec1=rec1/3
print("accuracy of my decision tree model is: ",acc1*100)
print("precision of my decision tree model is: ",prec1*100)
print("recall of my decision tree model is: ",rec1*100)
print('\n')

acc2=acc2/3
prec2=prec2/3
rec2=rec2/3
print("accuracy of sklearn decision tree model is: ",acc2*100)
print("precision of sklearn decision tree model is: ",prec2*100)
print("recall of sklearn decision tree model is: ",rec2*100)

confusion matrix of created model [[  0  18   0]
 [  2 436   8]
 [  0  56  13]]


confusion matrix of sklearn model [[  0  18   0]
 [  0 433  13]
 [  0  53  16]]


confusion matrix of created model [[  0  19   0]
 [  1 424   4]
 [  0  82   3]]


confusion matrix of sklearn model [[  0  19   0]
 [  0 428   1]
 [  0  69  16]]


confusion matrix of created model [[  0  25   1]
 [  1 417  26]
 [  0  49  14]]


confusion matrix of sklearn model [[  0  25   1]
 [  0 420  24]
 [  0  44  19]]


accuracy of my decision tree model is:  81.7385866166354
precision of my decision tree model is:  52.646222579792976
recall of my decision tree model is:  47.993396379237744


accuracy of sklearn decision tree model is:  83.30206378986867
precision of sklearn decision tree model is:  59.1262095813723
recall of sklearn decision tree model is:  51.196251137913116
